In [1]:
import pandas as pd
import requests
import plotly.graph_objects as go

In [2]:
def get_collection_info(collection_id):
    collection_info_response = requests.get(f"https://api.cellxgene.cziscience.com/dp/v1/collections/{collection_id}")
    collection_info = collection_info_response.json()
    return collection_info

In [6]:
# get data 
response = requests.get("https://api.cellxgene.cziscience.com/dp/v1/collections")
collections = response.json()["collections"]

data = [get_collection_info(c["id"]) for c in collections]

In [7]:
# create collection table 
collection_id = []
collection_dt = []
collection_name = []

for collection in data:
    collection_id.append(collection['id'])
    collection_dt.append(collection['created_at'])
    collection_name.append(collection['name'])

collections_df = pd.DataFrame({'collection_id': collection_id, 'collection_dt': collection_dt, 'collection_name': collection_name})
collections_df['collection_dt'] = pd.to_datetime(collections_df['collection_dt'] , unit='s')

In [8]:
# create dataset table 
keys_w_label = ['assay', 'development_stage', 'disease', 'ethnicity', 'tissue']
keys_wo_label = ['collection_id', 'id', 'name', 'sex', 'cell_count', 'created_at']
dataset_data = []

for c in data:
    for d in c['datasets']:
        keys = d.keys()
        keys_wo_labels_new = list(set(keys) & set(keys_wo_label))
        dict_updated = {key: d[key] for key in keys_wo_labels_new}
        keys_w_labels_new = list(set(keys) & set(keys_w_label))
        for key in keys_w_labels_new:
            dict_temp = {key: [i['label'] for i in d[key]]}
            dict_updated.update(dict_temp)
        dict_updated.update({'organism': d['organism']['label']})
        dataset_data.append(dict_updated)

datasets_df = pd.DataFrame(dataset_data)
datasets_df = datasets_df.rename(columns = {'id': 'dataset_id', 'name': 'dataset_name', 'created_at': 'dataset_dt'})
datasets_df['dataset_dt'] = pd.to_datetime(datasets_df['dataset_dt'], unit='s')

In [82]:
dataset_id_to_name = datasets_df[['dataset_name', 'dataset_id', 'dataset_dt']]

In [92]:
# downloads data
downloads_df = pd.read_csv('download_data.csv')

# remove downloads from datasets no longer on the platform
downloads_df = downloads_df[downloads_df['filetype'].notnull()].reset_index()

# remove downloads with bytessent == 1 
downloads_df = downloads_df[downloads_df['bytessent'] > 1].reset_index()

# remove downloads from myself(ip address = 71.200.124.31)
downloads_df = downloads_df[downloads_df['remoteip'] != '71.200.124.31']

# process datetime 
downloads_df['download_datetime'] = pd.to_datetime(downloads_df['download_datetime'], format='%d/%b/%Y:%H:%M:%S %z')
downloads_df['download_dt'] = downloads_df['download_datetime'].dt.date

downloads_df = downloads_df[['dataset_id', 'download_datetime', 'download_dt', 'filetype', 'remoteip', 'download_agent']].reset_index(drop = True)

downloads_df['total_downloads'] = 1

downloads_df = downloads_df.groupby(['dataset_id', 'download_dt', 'filetype', 'remoteip', 'download_agent']).sum().reset_index()


In [97]:
# limit to downloads of datasets on the platform 
combined = dataset_id_to_name.merge(downloads_df, how = 'inner', on = 'dataset_id')
combined['count'] = 1

# Single cell download metrics
- Version 1 - counts unique downloads over the following fields: remoteip, download_dt, filetype, dataset_id, download_agent
- Version 2 - counts daily unique dataset downloads by remoteip 

## Downloads over time


### Version 1

In [98]:
downloads_v1 = combined[['download_dt', 'count']].groupby(['download_dt']).sum().reset_index()
downloads_v1['cum_downloads'] = downloads_v1['count'].cumsum()

fig = go.Figure(go.Scatter(
    mode = "lines+markers",
    x = downloads_v1['download_dt'].tolist(),
    y = downloads_v1['cum_downloads'].tolist()
    ))

fig.update_layout(
    title="Number of dataset downloads over time",
    xaxis_title="date",
    yaxis_title="number of raw downloads"
)
             
fig.show()

### Version 2

In [148]:
downloads_v2 = combined[['download_dt', 'remoteip', 'dataset_id']].drop_duplicates()
downloads_v2['count'] = 1
downloads_v2 = downloads_v2[['download_dt', 'count']].groupby(['download_dt']).sum().reset_index()
downloads_v2['cum_downloads'] = downloads_v2['count'].cumsum()

fig = go.Figure(go.Scatter(
    mode = "lines+markers",
    x = downloads_v2['download_dt'].tolist(),
    y = downloads_v2['cum_downloads'].tolist()
    ))

fig.update_layout(
    title="Number of dataset downloads over time",
    xaxis_title="date",
    yaxis_title="number of unique downloads"
)
             
fig.show()

## Most downloaded datasets

### Version 1

In [101]:
top_ds_v1 = combined[['dataset_id', 'dataset_name', 'count']].groupby(['dataset_id', 'dataset_name']).sum().reset_index()
top_ds_v1= top_ds_v1.sort_values(by=['count'], ascending = False).reset_index(drop = True)
top_ds_v1[0:10]


,dataset_id,dataset_name,count
0,66d15835-5dc8-4e96-b0eb-f48971cb65e8,Single cell transcriptome analysis of human pa...,83
1,f72958f5-7f42-4ebb-98da-445b0c6de516,Azimuth meta-analysis of 10 datasets of health...,78
2,9df60c57-fdf3-4e93-828e-fe9303f20438,Single cell transcriptional and chromatin acce...,67
3,21d3e683-80a4-4d9b-bc89-ebb2df513dde,Time-resolved Systems Immunology Reveals a Lat...,62
4,13a027de-ea3e-432b-9a5e-6bc7048498fc,Single cell transcriptional and chromatin acce...,53
5,53d208b0-2cfd-4366-9866-c3c6114081bc,Tabula Sapiens - All Cells,40
6,b83559d1-156f-4ba9-9f6a-b165f83ef43f,Single-cell RNA-Seq Investigation of Foveal an...,38
7,b07e5164-baf6-43d2-bdba-5a249d0da879,A Single-Cell Transcriptome Atlas of the Human...,36
8,30cd5311-6c09-46c9-94f1-71fe4b91813c,Time-resolved Systems Immunology Reveals a Lat...,31
9,6cda3b13-7257-45b9-ac20-0a7e6697e4f2,scRNA-seq data analysis of HUVECs treated with...,31


### Version 2

In [217]:
top_ds_v2 = combined[['dataset_id', 'dataset_name','download_dt','remoteip']].drop_duplicates()
top_ds_v2['count'] = 1
top_ds_v2 = top_ds_v2[['dataset_id', 'dataset_name', 'count']].groupby(['dataset_id', 'dataset_name']).sum().reset_index()
top_ds_v2= top_ds_v2.sort_values(by=['count'], ascending = False).reset_index(drop = True)
top_ds_v2 = top_ds_v2[0:10]
#top_ds_v2['dataset_name'].iloc[4]
top_ds_v2

,dataset_id,dataset_name,count
0,f72958f5-7f42-4ebb-98da-445b0c6de516,Azimuth meta-analysis of 10 datasets of health...,66
1,66d15835-5dc8-4e96-b0eb-f48971cb65e8,Single cell transcriptome analysis of human pa...,65
2,9df60c57-fdf3-4e93-828e-fe9303f20438,Single cell transcriptional and chromatin acce...,60
3,21d3e683-80a4-4d9b-bc89-ebb2df513dde,Time-resolved Systems Immunology Reveals a Lat...,53
4,13a027de-ea3e-432b-9a5e-6bc7048498fc,Single cell transcriptional and chromatin acce...,47
5,b83559d1-156f-4ba9-9f6a-b165f83ef43f,Single-cell RNA-Seq Investigation of Foveal an...,36
6,b07e5164-baf6-43d2-bdba-5a249d0da879,A Single-Cell Transcriptome Atlas of the Human...,35
7,53d208b0-2cfd-4366-9866-c3c6114081bc,Tabula Sapiens - All Cells,34
8,6cda3b13-7257-45b9-ac20-0a7e6697e4f2,scRNA-seq data analysis of HUVECs treated with...,29
9,30cd5311-6c09-46c9-94f1-71fe4b91813c,Time-resolved Systems Immunology Reveals a Lat...,29


### Version 3

In [154]:
top_ds_v3 = combined[['dataset_id', 'dataset_name', 'remoteip']].drop_duplicates()
top_ds_v3['count'] = 1
top_ds_v3 = top_ds_v3[['dataset_id', 'dataset_name', 'count']].groupby(['dataset_id', 'dataset_name']).sum().reset_index()
top_ds_v3= top_ds_v3.sort_values(by=['count'], ascending = False).reset_index(drop = True)
top_ds_v3[0:10]

,dataset_id,dataset_name,count
0,f72958f5-7f42-4ebb-98da-445b0c6de516,Azimuth meta-analysis of 10 datasets of health...,57
1,9df60c57-fdf3-4e93-828e-fe9303f20438,Single cell transcriptional and chromatin acce...,47
2,21d3e683-80a4-4d9b-bc89-ebb2df513dde,Time-resolved Systems Immunology Reveals a Lat...,47
3,66d15835-5dc8-4e96-b0eb-f48971cb65e8,Single cell transcriptome analysis of human pa...,42
4,13a027de-ea3e-432b-9a5e-6bc7048498fc,Single cell transcriptional and chromatin acce...,37
5,53d208b0-2cfd-4366-9866-c3c6114081bc,Tabula Sapiens - All Cells,32
6,b83559d1-156f-4ba9-9f6a-b165f83ef43f,Single-cell RNA-Seq Investigation of Foveal an...,26
7,30cd5311-6c09-46c9-94f1-71fe4b91813c,Time-resolved Systems Immunology Reveals a Lat...,26
8,d4e69e01-3ba2-4d6b-a15d-e7048f78f22e,All — Cells of the adult human heart,24
9,b07e5164-baf6-43d2-bdba-5a249d0da879,A Single-Cell Transcriptome Atlas of the Human...,23


## Download patterns of top datasets

In [142]:
def create_trace(dataset_id):
    output = downloads_df[downloads_df['dataset_id'] == dataset_id]
    output = output.groupby(['download_dt','filetype', 'remoteip', 'download_agent']).sum().reset_index()
    output['count'] = 1
    output['download_dt'] = pd.to_datetime(output['download_dt'])
    output['download_year_month'] = output['download_dt'].dt.strftime('%Y-%m')
    output = output[['download_year_month', 'count']].groupby(['download_year_month']).sum().reset_index()
    return (output['download_year_month'].tolist(), output['count'].tolist())
    

In [86]:
azimuth = downloads_df[downloads_df['dataset_id'] == 'f72958f5-7f42-4ebb-98da-445b0c6de516']
azimuth = azimuth.groupby(['download_dt','filetype', 'remoteip', 'download_agent']).sum().reset_index()
azimuth['count'] = 1
azimuth = azimuth[['download_dt', 'count']].groupby(['download_dt']).sum().reset_index()

fig = go.Figure(go.Scatter(
    mode = "lines+markers",
    x = azimuth['download_dt'].tolist(),
    y = azimuth['count'].tolist()
    ))

fig.update_layout(
    title="Number of Azimuth meta-analysis downloads over time",
    xaxis_title="date",
    yaxis_title="number of downloads"
)
             
fig.show()

In [162]:
# add traces
top1_x, top1_y = create_trace('f72958f5-7f42-4ebb-98da-445b0c6de516')
top2_x, top2_y = create_trace('66d15835-5dc8-4e96-b0eb-f48971cb65e8')
top3_x, top3_y = create_trace('9df60c57-fdf3-4e93-828e-fe9303f20438')
top4_x, top4_y = create_trace('21d3e683-80a4-4d9b-bc89-ebb2df513dde')
top5_x, top5_y = create_trace('13a027de-ea3e-432b-9a5e-6bc7048498fc')

In [176]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=top1_x, y=top1_y,
                    mode='lines+markers',
                    name='Azimuth meta-analysis'))

fig.add_trace(go.Scatter(x=top2_x, y=top2_y,
                    mode='lines+markers',
                    name='Single cell transcriptome analysis'))

fig.add_trace(go.Scatter(x=top3_x, y=top3_y,
                    mode='lines+markers',
                    name='Single cell transcriptional and chromatin accessibility profiling - RNAseq'))

fig.add_trace(go.Scatter(x=top4_x, y=top4_y,
                    mode='lines+markers',
                    name='Time-resolved Systems Immunology'))

fig.add_trace(go.Scatter(x=top5_x, y=top5_y,
                    mode='lines+markers',
                    name='Single cell transcriptional and chromatin accessibility - ATACseq'))

fig.update_layout(
    title="Trends of top dataset downloads by month",
    xaxis_title="date",
    yaxis_title="number of downloads",
    legend = dict(y = -0.8, x = 0)
)


## Downloads by dataset characteristic

In [213]:
df_char = downloads_df[['dataset_id', 'download_dt', 'filetype', 'remoteip', 'download_agent']]
df_char['count'] = 1
df_char = df_char.merge(datasets_df, how = 'inner', on = 'dataset_id')

In [214]:
ds_ethnicity = datasets_df[['ethnicity']].explode('ethnicity').reset_index(drop = True) 
ds_ethnicity['total'] = 1
ds_ethnicity = ds_ethnicity.groupby(['ethnicity']).sum().reset_index()

In [215]:
df_ethnicity = df_char[['ethnicity', 'count']].explode('ethnicity').reset_index(drop = True)
df_ethnicity = df_ethnicity.groupby(['ethnicity']).sum().reset_index()

df_ethnicity = df_ethnicity.merge(ds_ethnicity, how = 'inner', on = 'ethnicity')
df_ethnicity['normalized'] = round(df_ethnicity['count']/df_ethnicity['total'])
df_ethnicity

,ethnicity,count,total,normalized
0,African American,178,8,22.0
1,Asian,98,8,12.0
2,Chinese,29,1,29.0
3,East Asian,93,2,46.0
4,European,452,20,23.0
5,Finnish,23,1,23.0
6,Hispanic or Latin American,225,6,38.0
7,male,83,1,83.0
8,na,457,68,7.0
9,unknown,1191,77,15.0


In [216]:
fig = go.Figure([go.Bar(x=df_ethnicity['ethnicity'].tolist(), y=df_ethnicity['normalized'].tolist(), 
                        text=df_ethnicity['normalized'].tolist(), textposition='outside')])

fig.show()